In [11]:
"""Module to check energy of ligand conformations."""

from __future__ import annotations

import logging
from copy import deepcopy
from functools import lru_cache

import numpy as np
from rdkit import ForceField  # noqa: F401
from rdkit.Chem.inchi import InchiReadWriteError, MolFromInchi, MolToInchi
from rdkit.Chem.rdchem import Mol
from rdkit.Chem.rdDistGeom import EmbedMultipleConfs, ETKDGv3
from rdkit.Chem.rdForceFieldHelpers import (
    UFFGetMoleculeForceField,
    UFFOptimizeMoleculeConfs,
)
from rdkit.Chem.rdmolops import AddHs, AssignStereochemistryFrom3D, SanitizeMol

In [12]:
def check_energy_ratio(
    mol_pred: Mol,
    threshold_energy_ratio: float = 7.0,
    ensemble_number_conformations: int = 100,
    inchi_strict: bool = False,
):
    """Check whether the energy of the docked ligand is within user defined range.

    Args:
        mol_pred: Predicted molecule (docked ligand) with exactly one conformer.
        threshold_energy_ratio: Limit above which the energy ratio is deemed to high. Defaults to 7.0.
        ensemble_number_conformations: Number of conformations to generate for the ensemble over which to
            average. Defaults to 100.
        inchi_strict: Whether to treat warnings in the InChI generation as errors. Defaults to False.

    Returns:
        PoseBusters results dictionary.
    """
    mol_pred = deepcopy(mol_pred)
    assert mol_pred.GetNumConformers() > 0, "Molecule does not have a conformer."
    mol_pred = assert_sanity(mol_pred)
    AddHs(mol_pred, addCoords=True)

    inchi = get_inchi(mol_pred, inchi_strict=inchi_strict)

    try:
        conf_energy = get_conf_energy(mol_pred)
    except Exception:
        conf_energy = np.nan
    try:
        avg_energy = float(get_average_energy(inchi, ensemble_number_conformations))
    except Exception:
        avg_energy = 0
    if avg_energy == 0:
        avg_energy = np.nan

    pred_factor = conf_energy / avg_energy
    ratio_passes = pred_factor <= threshold_energy_ratio

    results = {
        "ensemble_avg_energy": avg_energy,
        "mol_pred_energy": conf_energy,
        "energy_ratio": pred_factor,
        "energy_ratio_passes": ratio_passes,
    }
    return results


def get_average_energy(inchi: str, n_confs: int = 50, num_threads: int = 0) -> Mol:
    """Get average energy of an ensemble of molecule conformations."""
    mol = MolFromInchi(inchi)
    energies = new_conformation(mol, n_confs, num_threads)["energies"]
    return sum(energies) / len(energies)

def get_conf_energy(mol: Mol, conf_id: int = -1) -> float:
    """Get energy of a conformation."""
    assert mol is not None

    mol = AddHs(mol, addCoords=True)
    uff = UFFGetMoleculeForceField(mol, confId=conf_id)
    e_uff = uff.CalcEnergy()
    return e_uff

def get_inchi(mol: Mol, inchi_strict: bool = False) -> str:
    """Get inchi of a molecule."""

    SanitizeMol(mol)
    inchi = MolToInchi(mol, treatWarningAsError=inchi_strict)
    # check inchi because inchi generation does not raise an error if the inchi is invalid
    return inchi

def assert_sanity(mol: Mol) -> Mol:
    """Check that RDKit sanitization does not fail."""
    flags = SanitizeMol(mol)
    assert flags == 0, f"Sanitization failed with flags {flags}"
    return mol

def new_conformation(mol: Mol, n_confs: int = 1, num_threads: int = 0, energy_minimization=True) -> Mol:
    """Generate new conformation(s) for a molecule."""
    assert mol is not None

    etkdg = ETKDGv3()
    etkdg.randomSeed = 42
    etkdg.verbose = False
    etkdg.useRandomCoords = True
    etkdg.numThreads = num_threads

    # prep mol
    mol_etkdg = deepcopy(mol)
    mol_etkdg = AddHs(mol_etkdg, addCoords=True)
    AssignStereochemistryFrom3D(mol_etkdg, replaceExistingTags=True)
    mol_etkdg.RemoveAllConformers()

    # etkdg
    cids_etkdg = EmbedMultipleConfs(mol_etkdg, n_confs, etkdg)
    assert len(cids_etkdg) == n_confs, "Failed to generate conformations."

    # energy minimization
    if energy_minimization:
        energy_uff = UFFOptimizeMoleculeConfs(mol_etkdg, numThreads=num_threads)
        energy_uff = [v[1] for v in energy_uff]
    else:
        energy_uff = [get_conf_energy(mol_etkdg, conf_id) for conf_id in cids_etkdg]

    return {"mol": mol_etkdg, "energies": energy_uff}

In [ ]:
# gcdm
from rdkit import Chem
import csv, random
import glob, os, torch
from rdkit import Chem, Geometry
atom_encoder= {"H": 0, "C": 1, "N": 2, "O": 3, "F": 4}
atom_decoder= ["H", "C", "N", "O", "F"]
def load_molecule_xyz(
    file
):
    with open(file, encoding="utf8") as f:
        num_atoms = int(f.readline())
        one_hot = torch.zeros(num_atoms, len(atom_decoder))
        positions = torch.zeros(num_atoms, 3)
        f.readline()
        atoms = f.readlines()
        for i in range(num_atoms):
            atom = atoms[i].split(" ")
            atom_type = atom[0]
            one_hot[i, atom_encoder[atom_type]] = 1
            position = torch.Tensor([float(e) for e in atom[1:]])
            positions[i, :] = position
        return positions, one_hot

def load_files_with_ext(path, ext, shuffle=True):
    files = glob.glob(path + f"/*.{ext}")
    if shuffle:
        random.shuffle(files)
    return files
bond_dict = [None, Chem.rdchem.BondType.SINGLE, Chem.rdchem.BondType.DOUBLE, Chem.rdchem.BondType.TRIPLE,
             Chem.rdchem.BondType.AROMATIC]
bonds1 = {'H': {'H': 74, 'C': 109, 'N': 101, 'O': 96, 'F': 92,
                'B': 119, 'Si': 148, 'P': 144, 'As': 152, 'S': 134,
                'Cl': 127, 'Br': 141, 'I': 161},
          'C': {'H': 109, 'C': 154, 'N': 147, 'O': 143, 'F': 135,
                'Si': 185, 'P': 184, 'S': 182, 'Cl': 177, 'Br': 194,
                'I': 214},
          'N': {'H': 101, 'C': 147, 'N': 145, 'O': 140, 'F': 136,
                'Cl': 175, 'Br': 214, 'S': 168, 'I': 222, 'P': 177},
          'O': {'H': 96, 'C': 143, 'N': 140, 'O': 148, 'F': 142,
                'Br': 172, 'S': 151, 'P': 163, 'Si': 163, 'Cl': 164,
                'I': 194},
          'F': {'H': 92, 'C': 135, 'N': 136, 'O': 142, 'F': 142,
                'S': 158, 'Si': 160, 'Cl': 166, 'Br': 178, 'P': 156,
                'I': 187},
          'B': {'H': 119, 'Cl': 175},
          'Si': {'Si': 233, 'H': 148, 'C': 185, 'O': 163, 'S': 200,
                 'F': 160, 'Cl': 202, 'Br': 215, 'I': 243},
          'Cl': {'Cl': 199, 'H': 127, 'C': 177, 'N': 175, 'O': 164,
                 'P': 203, 'S': 207, 'B': 175, 'Si': 202, 'F': 166,
                 'Br': 214},
          'S': {'H': 134, 'C': 182, 'N': 168, 'O': 151, 'S': 204,
                'F': 158, 'Cl': 207, 'Br': 225, 'Si': 200, 'P': 210,
                'I': 234},
          'Br': {'Br': 228, 'H': 141, 'C': 194, 'O': 172, 'N': 214,
                 'Si': 215, 'S': 225, 'F': 178, 'Cl': 214, 'P': 222},
          'P': {'P': 221, 'H': 144, 'C': 184, 'O': 163, 'Cl': 203,
                'S': 210, 'F': 156, 'N': 177, 'Br': 222},
          'I': {'H': 161, 'C': 214, 'Si': 243, 'N': 222, 'O': 194,
                'S': 234, 'F': 187, 'I': 266},
          'As': {'H': 152}
          }

bonds2 = {'C': {'C': 134, 'N': 129, 'O': 120, 'S': 160},
          'N': {'C': 129, 'N': 125, 'O': 121},
          'O': {'C': 120, 'N': 121, 'O': 121, 'P': 150},
          'P': {'O': 150, 'S': 186},
          'S': {'P': 186}}

bonds3 = {'C': {'C': 120, 'N': 116, 'O': 113},
          'N': {'C': 116, 'N': 110},
          'O': {'C': 113}}

margin1, margin2, margin3 = 5, 2, 1

def set_positions(rd_mol, pos):
    n_atoms = len(rd_mol.GetAtoms())
    rd_conf = Chem.Conformer(n_atoms)
    xyz = pos
    # add atoms and coordinates
    for i in range(n_atoms):
        rd_coords = Geometry.Point3D(xyz[i][0].item(), xyz[i][1].item(), xyz[i][2].item())
        rd_conf.SetAtomPosition(i, rd_coords)
    rd_mol.AddConformer(rd_conf)
    return rd_mol

def build_molecule(positions, atom_types):
    X, A, E = build_xae_molecule(positions, atom_types)
    mol = Chem.RWMol()
    for atom in X:
        a = Chem.Atom(atom_decoder[atom])
        mol.AddAtom(a)

    all_bonds = torch.nonzero(A)
    for bond in all_bonds:
        mol.AddBond(bond[0].item(), bond[1].item(), bond_dict[E[bond[0], bond[1]].item()])
    mol = mol.GetMol()
    mol = set_positions(mol, positions)
    return mol


def build_xae_molecule(positions, atom_types):
    """ Returns a triplet (X, A, E): atom_types, adjacency matrix, edge_types
        args:
        positions: N x 3  (already masked to keep final number nodes)
        atom_types: N
        returns:
        X: N         (int)
        A: N x N     (bool)                  (binary adjacency matrix)
        E: N x N     (int)  (bond type, 0 if no bond) such that A = E.bool()
    """
    # atom_decoder = dataset_info['atom_decoder']
    n = positions.shape[0]
    X = atom_types
    A = torch.zeros((n, n), dtype=torch.bool)
    E = torch.zeros((n, n), dtype=torch.int)

    pos = positions.unsqueeze(0)
    dists = torch.cdist(pos, pos, p=2).squeeze(0)
    for i in range(n):
        for j in range(i):
            pair = sorted([atom_types[i], atom_types[j]])
            order = geom_predictor((atom_decoder[pair[0]], atom_decoder[pair[1]]), dists[i, j],
                                       limit_bonds_to_one=False)
                # order = get_bond_order(atom_decoder[pair[0]], atom_decoder[pair[1]], dists[i, j])

            # TODO: a batched version of get_bond_order to avoid the for loop
            if order > 0:
                # Warning: the graph should be DIRECTED
                A[i, j] = 1
                E[i, j] = order
    return X, A, E

def geom_predictor(p, l, margin1=5, limit_bonds_to_one=False):
    """ p: atom pair (couple of str)
        l: bond length (float)"""
    bond_order = get_bond_order(p[0], p[1], l, check_exists=True)

    # If limit_bonds_to_one is enabled, every bond type will return 1.
    if limit_bonds_to_one:
        return 1 if bond_order > 0 else 0
    else:
        return bond_order
    
def get_bond_order(atom1, atom2, distance, check_exists=False):
    distance = 100 * distance  # We change the metric

    # Check exists for large molecules where some atom pairs do not have a
    # typical bond length.
    if check_exists:
        if atom1 not in bonds1:
            return 0
        if atom2 not in bonds1[atom1]:
            return 0

    # margin1, margin2 and margin3 have been tuned to maximize the stability of
    # the QM9 true samples.
    if distance < bonds1[atom1][atom2] + margin1:

        # Check if atoms in bonds2 dictionary.
        if atom1 in bonds2 and atom2 in bonds2[atom1]:
            thr_bond2 = bonds2[atom1][atom2] + margin2
            if distance < thr_bond2:
                if atom1 in bonds3 and atom2 in bonds3[atom1]:
                    thr_bond3 = bonds3[atom1][atom2] + margin3
                    if distance < thr_bond3:
                        return 3  # Triple
                return 2  # Double
        return 1  # Single
    return 0  # No bond
root_dir = '/home/yyw/yyw/bio-diffusion/logs/train/runs/2024-09-04_11-27-20/'
path = glob.glob(os.path.join(root_dir, 'epoch_*'))
ee=[]
for p in path:
    # condition_folder = os.path.join(p, 'conditional')
    if os.path.exists(p):
        files = load_files_with_ext(p, ext="xyz")
        for file in files:
            positions, one_hot = load_molecule_xyz(file)
            atom_type = torch.argmax(one_hot, dim=1).numpy()
            mol = build_molecule(positions, atom_type)
            ee.append(mol)
with open('output_gcdm_3d_energy_qm9.csv', mode='w', newline='') as file:
    # Initialize writer (fieldnames will depend on your dictionary keys)
    fieldnames = ["ensemble_avg_energy","mol_pred_energy","energy_ratio","energy_ratio_passes"]  # Change this to the actual keys you expect
    writer = csv.DictWriter(file, fieldnames=fieldnames)

    # Write the header
    writer.writeheader()
    # for i in range(100):
    #     suppl = Chem.SDMolSupplier(f'/home/yyw/yyw/GCDM-SBDD/results/8tzf_mol_{i}.sdf')
    #     for mol in suppl:
            # mol_frags = Chem.rdmolops.GetMolFrags(mol, asMols=True, sanitizeFrags=False)
            # largest_mol = max(mol_frags, key=lambda m: m.GetNumAtoms())
    for mol in ee:
        try:
            mol = assert_sanity(mol)
        except Exception as e:
            continue
        energy_dict = check_energy_ratio(mol)
        writer.writerow(energy_dict)

In [ ]:
# DDMCM
import os
import rdkit
from rdkit import Chem
from rdkit.Chem import Draw, AllChem
import numpy as np
from rdkit.Geometry import Point3D
# from src.analysis.rdkit_functions import correct_mol
import torch
bond_dict = [None, Chem.rdchem.BondType.SINGLE, Chem.rdchem.BondType.DOUBLE, Chem.rdchem.BondType.TRIPLE,
             Chem.rdchem.BondType.AROMATIC]
atom_decoder = ['C', 'N', 'S', 'O', 'F', 'Cl', 'Br']

def mol_from_graphs(node_list, adjacency_matrix, positions, max_ring_size=6):
        """
        Convert graphs to rdkit molecules
        node_list: the nodes of a batch of nodes (bs x n)
        adjacency_matrix: the adjacency_matrix of the molecule (bs x n x n)
        """
        # dictionary to map integer value to the char of atom

        # create empty editable mol object
        mol = Chem.RWMol()
        node_to_idx = {}
        # add atoms to mol and keep track of index
        for i in range(len(node_list)):
            if node_list[i] == -1:
                continue
            a = Chem.Atom(atom_decoder[int(node_list[i])])
            molIdx = mol.AddAtom(a)
            node_to_idx[i] = molIdx
            
        edge_types = torch.triu(adjacency_matrix, diagonal=1)
        edge_types[edge_types == -1] = 0
        all_bonds = torch.nonzero(edge_types)
        for i, bond in enumerate(all_bonds):
            if bond[0].item() != bond[1].item():
                mol.AddBond(bond[0].item(), bond[1].item(), bond_dict[int(edge_types[bond[0], bond[1]].item())])
        
        # Set coordinates
        # positions = positions.double()
        try:
            mol = mol.GetMol()
        except rdkit.Chem.KekulizeException:
            print("Can't kekulize molecule")
            return None
        
        if positions is not None:
            if np.isnan(np.array(positions)).any():  #直接扔掉
                return mol
            else:
                conf = Chem.Conformer(mol.GetNumAtoms())   #可能顺序不一样
                for i in range(mol.GetNumAtoms()):
                    conf.SetAtomPosition(i, Point3D(positions[i][0], positions[i][1], positions[i][2]))
                mol.AddConformer(conf)
                # for i, p in enumerate(positions):
                #     if i in node_to_idx:
                #         molIdx = node_to_idx[i]
                #         conf.SetAtomPosition(molIdx, Point3D(p[0].item(), p[1].item(), p[2].item()))
                #     else:
                #         continue
                # mol.AddConformer(conf)

        return mol
all_mols = []
smiles = []
file = '/raid/yyw/PharmDiGress/outputs/2024-10-21/16-13-22-moses_dual_glp1/generated_samples2.txt'
with open(file, encoding="utf8") as f:
    lines = f.readlines()
i=0
while i < len(lines):
    line = lines[i].strip()
    if line.startswith("N"):
        N=int(line.split("=")[1])
        i+=1
        if lines[i].strip().startswith("X"):
            X = lines[i + 1].strip().split()
            i += 2
        pos_content = []
        if lines[i].strip().startswith("pos"):
            for _ in range(N):
                i += 1
                pos_content.append(list(map(float,lines[i].strip().split())))
        i+=1
        E_matrix = []
        if lines[i].strip().startswith("E"):
            for _ in range(N):
                i += 1
                E_row = list(map(int, lines[i].strip().split()))
                E_matrix.append(E_row)
            E_matrix=torch.tensor(E_matrix)
    else:
        i+=1
    rdmol = mol_from_graphs(X, E_matrix, pos_content)
    try:
        mol_frags = Chem.rdmolops.GetMolFrags(rdmol, asMols=True, sanitizeFrags=False)
        largest_mol = max(mol_frags, default=rdmol, key=lambda m: m.GetNumAtoms())
        Chem.SanitizeMol(largest_mol)
        all_mols.append(largest_mol)
        smiles.append(Chem.MolToSmiles(largest_mol))
    except Chem.rdchem.AtomValenceException:
        print("Valence error in GetmolFrags")
    except Chem.rdchem.KekulizeException:
        print("Can't kekulize molecule")
    except Chem.rdchem.AtomKekulizeException or ValueError:
        print("other error")
    


[14:11:33] Can't kekulize mol.  Unkekulized atoms: 6 7 8 13 19
[14:11:33] Can't kekulize mol.  Unkekulized atoms: 6 7 8 13 19
[14:11:33] Can't kekulize mol.  Unkekulized atoms: 6 7 8 13 19
[14:11:33] Can't kekulize mol.  Unkekulized atoms: 6 7 8 13 19
[14:11:33] Explicit valence for atom # 5 N, 6, is greater than permitted
[14:11:33] Explicit valence for atom # 5 N, 6, is greater than permitted
[14:11:33] Explicit valence for atom # 5 N, 6, is greater than permitted
[14:11:33] Explicit valence for atom # 5 N, 6, is greater than permitted
[14:11:33] Explicit valence for atom # 3 C, 5, is greater than permitted
[14:11:33] Explicit valence for atom # 3 C, 5, is greater than permitted
[14:11:33] Explicit valence for atom # 3 C, 5, is greater than permitted
[14:11:33] Explicit valence for atom # 3 C, 5, is greater than permitted
[14:11:33] Explicit valence for atom # 1 N, 5, is greater than permitted
[14:11:33] Explicit valence for atom # 1 N, 5, is greater than permitted
[14:11:33] Explici

Can't kekulize molecule
Can't kekulize molecule
Can't kekulize molecule
Can't kekulize molecule
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Can't kekulize molecule
Can't kekulize molecule
Can't kekulize molecule
Can't kekulize molecule
Can't kekulize molecule
Can't kekulize molecule
Can't kekulize molecule
Can't kekulize molecule
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
other error
other error
other error
other error


[14:11:33] non-ring atom 1 marked aromatic
[14:11:33] non-ring atom 1 marked aromatic
[14:11:33] non-ring atom 1 marked aromatic
[14:11:33] non-ring atom 1 marked aromatic
[14:11:33] Explicit valence for atom # 18 C, 5, is greater than permitted
[14:11:33] Explicit valence for atom # 18 C, 5, is greater than permitted
[14:11:33] Explicit valence for atom # 18 C, 5, is greater than permitted
[14:11:33] Explicit valence for atom # 18 C, 5, is greater than permitted
[14:11:33] Explicit valence for atom # 25 O, 3, is greater than permitted
[14:11:33] Explicit valence for atom # 25 O, 3, is greater than permitted
[14:11:33] Explicit valence for atom # 25 O, 3, is greater than permitted
[14:11:33] Explicit valence for atom # 25 O, 3, is greater than permitted
[14:11:33] Can't kekulize mol.  Unkekulized atoms: 2 17 22 23 24
[14:11:33] Can't kekulize mol.  Unkekulized atoms: 2 17 22 23 24
[14:11:33] Can't kekulize mol.  Unkekulized atoms: 2 17 22 23 24
[14:11:33] Can't kekulize mol.  Unkekuliz

other error
other error
other error
other error
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Can't kekulize molecule
Can't kekulize molecule
Can't kekulize molecule
Can't kekulize molecule
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags


[14:11:33] Explicit valence for atom # 25 C, 5, is greater than permitted
[14:11:33] Explicit valence for atom # 25 C, 5, is greater than permitted
[14:11:33] Explicit valence for atom # 25 C, 5, is greater than permitted
[14:11:33] Can't kekulize mol.  Unkekulized atoms: 0 1 11 13
[14:11:33] Can't kekulize mol.  Unkekulized atoms: 0 1 11 13
[14:11:33] Can't kekulize mol.  Unkekulized atoms: 0 1 11 13
[14:11:33] Can't kekulize mol.  Unkekulized atoms: 0 1 11 13
[14:11:33] Explicit valence for atom # 7 C, 5, is greater than permitted
[14:11:33] Explicit valence for atom # 7 C, 5, is greater than permitted
[14:11:33] Explicit valence for atom # 7 C, 5, is greater than permitted
[14:11:33] Explicit valence for atom # 7 C, 5, is greater than permitted
[14:11:33] Explicit valence for atom # 11 C, 6, is greater than permitted
[14:11:33] Explicit valence for atom # 11 C, 6, is greater than permitted
[14:11:33] Explicit valence for atom # 11 C, 6, is greater than permitted
[14:11:33] Explicit 

...

Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
other error
other error
other error
other error
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Can't kekulize molecule
Can't kekulize molecule
Can't kekulize molecule
Can't kekulize molecule
Can't kekulize molecule
Can't kekulize molecule
Can't kekulize molecule
Can't kekulize molecule
Can't kekulize molecule
Can't kekulize molecule
Can't kekulize molecule
Can't kekulize molecule
other error
other error
other error
other error
other error
other error
other error
other error
Valence error in 

[14:12:14] Can't kekulize mol.  Unkekulized atoms: 0 1 6 11 12 14 15 19 20 22 23
[14:12:14] Explicit valence for atom # 9 C, 5, is greater than permitted
[14:12:14] Explicit valence for atom # 9 C, 5, is greater than permitted
[14:12:14] Explicit valence for atom # 9 C, 5, is greater than permitted
[14:12:14] Explicit valence for atom # 9 C, 5, is greater than permitted
[14:12:14] Explicit valence for atom # 8 F, 2, is greater than permitted
[14:12:14] Explicit valence for atom # 8 F, 2, is greater than permitted
[14:12:14] Explicit valence for atom # 8 F, 2, is greater than permitted
[14:12:14] Explicit valence for atom # 8 F, 2, is greater than permitted
[14:12:14] Explicit valence for atom # 7 C, 7, is greater than permitted
[14:12:14] Explicit valence for atom # 7 C, 7, is greater than permitted
[14:12:14] Explicit valence for atom # 7 C, 7, is greater than permitted
[14:12:14] Explicit valence for atom # 7 C, 7, is greater than permitted
[14:12:14] Explicit valence for atom # 5 C,

Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
other error
other error
other error
other error
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolF

[14:12:15] Explicit valence for atom # 18 O, 3, is greater than permitted
[14:12:15] non-ring atom 3 marked aromatic
[14:12:15] non-ring atom 3 marked aromatic
[14:12:15] non-ring atom 3 marked aromatic
[14:12:15] non-ring atom 3 marked aromatic
[14:12:15] Explicit valence for atom # 11 C, 7, is greater than permitted
[14:12:15] Explicit valence for atom # 11 C, 7, is greater than permitted
[14:12:15] Explicit valence for atom # 11 C, 7, is greater than permitted
[14:12:15] Explicit valence for atom # 11 C, 7, is greater than permitted
[14:12:15] Explicit valence for atom # 2 F, 2, is greater than permitted
[14:12:15] Explicit valence for atom # 2 F, 2, is greater than permitted
[14:12:15] Explicit valence for atom # 2 F, 2, is greater than permitted
[14:12:15] Explicit valence for atom # 2 F, 2, is greater than permitted


other error
other error
other error
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
Valence error in GetmolFrags
other error
other error
other error
other error


In [8]:
with Chem.SDWriter('glp1_moses_output.sdf') as writer:
    for mol in all_mols:
        try:
            Chem.SanitizeMol(mol)
            writer.write(mol)
        except:
            continue

In [ ]:
from rdkit import Chem
SDMols = Chem.SDMolSupplier('glp1_moses_output.sdf')
SDMol = [m for m in SDMols if m is not None]
smiles = [Chem.MolToSmiles(m) for m in SDMol]
from mini_moses import get_all_metrics
get_all_metrics(smiles)

In [1]:
smiles

NameError: name 'smiles' is not defined

In [2]:
# len(test)=12804
import random
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Draw, AllChem
df = pd.read_csv('/raid/yyw/PharmDiGress/data/moses/moses_pyg/raw/test_moses.csv')
smiles = random.sample(df['smiles'].to_list(), 941)
moses_test=[]
for s in smiles:
    mol = Chem.MolFromSmiles(s)
    m = Chem.AddHs(mol)
    try:
        AllChem.EmbedMolecule(m, useRandomCoords=True)
        AllChem.MMFFOptimizeMolecule(m)
    except ValueError:
        continue
    moses_test.append(m)
    

In [ ]:
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import AllChem
import numpy as np
# Calculate USRCAT fingerprint
def calculate_USRCAT_fingerprints(mols_list):
    fingerprints=[]
    for mol in mols_list:
        try:
            fg = rdMolDescriptors.GetUSRCAT(m)
        except ValueError:
            atom_rings=m.GetRingInfo().AtomRings()  
            large_rings = [ring for ring in atom_rings if len(ring) > 6]

            # 复制分子以避免修改原分子
            mol_edit = Chem.RWMol(mol)
            AllChem.EmbedMolecule(mol_edit)
            AllChem.MMFFOptimizeMolecule(mol_edit)
            for ring in large_rings:
                # 打断大环中的一个键
                mol_edit.RemoveBond(ring[0], ring[1])
            
            fg = rdMolDescriptors.GetUSRCAT(m)
        fingerprints.append(fg)
    # fingerprints = [rdMolDescriptors.GetUSRCAT(mol) for mol in mols_list]
    # Convert fingerprints to numpy array
    fingerprint_array = np.array([np.array(fp) for fp in fingerprints])
    return fingerprint_array
fingerprint_array = calculate_USRCAT_fingerprints(full)
from sklearn.preprocessing import StandardScaler
usrcat_array = StandardScaler().fit_transform(fingerprint_array)
from sklearn.manifold import TSNE
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2)
tsne_results = tsne.fit_transform(usrcat_array)
X_tsne = pd.DataFrame(usrcat_array, columns=[f'FP_{i}' for i in range(usrcat_array.shape[1])])
X_tsne['TSNE_1'] = tsne_results[:, 0]
X_tsne['TSNE_2'] = tsne_results[:, 1]
X_tsne['label'] = ['DDMCM'] * 941 + ['Test set'] * 941
plt.figure(figsize=(8,6))
X_tsne_subset = X_tsne[['TSNE_1', 'TSNE_2', 'label']]
sns.pairplot(X_tsne_subset, hue="label")
plt.savefig('TSNE_moses_output_usrcat.svg', dpi=600)
plt.show()